# first attempt at solving rooms with cpm

given the following:

Imagine you are designing a hospital patient floor. here's the rules:

the floorplate size will be 860 sqft per bed

the basis of design is from a hospital patient floor with 24 beds with the floorplate as 20640 sqft

there's six room types: patient_room, ada_room, supply, soiled, norish, and alcove

from the original design, we derive the ratio of each room type to beds, as well as a default size for each room type. it looks like this:

|Name of room|	# / Bed|	Default SQFT/Room|
|---|---|---|
|patient_room|	0.916666667|	792.9969057|
|ada_room|	0.083333333|	928.447425|
|supply|	0.041666667|	472.7116285|
|soiled|	0.041666667|	396.9733013|
|nourish|	0.041666667|	237.6616475|
|alcove|	0.125|	76.6088827|

now we need to fit these rooms to a patient floor with 40 beds. we say the number of each room type will be according to the ratio, and the sqft of the floorplate will be 34400. 

however, some of the room sizes will have to be adjusted. so we say the numbers of each type of room in our new taget patient floor, and the constarints on the size changes are as follows:

|Name of room|	number|	min sqft|	default sqft|	max sqft|
|---|---|---|---|---|
|patient_room|	37	|634|	793|	952|
|ada_room|	3|	742|	928|	1114|
|supply|	2|	378|	473|	568|
|soiled|	2|	318|	397|	476|
|nourish|	2|	190|	238|	286|
|alcove|	5|	62|	77|	92|




In [1]:
# !pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('optimize rooms', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

here's our constants

In [2]:
n_patient_room = 37
n_ada_room = 3
n_supply = 2
n_soiled = 2
n_nourish = 2
n_alcove = 2
d_patient_room = 793
d_ada_room = 928
d_supply = 473
d_soiled = 397
d_nourish = 238
d_alcove = 77

and here's the variables

In [3]:
s_patient_room = solver.IntVar(753, 833, 'patient_room')
s_ada_room = solver.IntVar(882, 974, 'ada_room')
s_supply = solver.IntVar(449, 497, 'supply')
s_soiled = solver.IntVar(377, 417, 'soiled')
s_nourish = solver.IntVar(226, 250, 'nourish')
s_alcove = solver.IntVar(73, 81, 'alcove')

add constraints like so

In [4]:
solver.Add(s_patient_room*n_patient_room + s_ada_room*n_ada_room + s_supply*n_supply + s_soiled*n_soiled + s_nourish*n_nourish + s_alcove*n_alcove <= 34400) # fit to floorplate

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001D03AFD3E40> >

now we need the objective function. in this case, we are minimizing the product of the [sum of the differences in all room sizes compared to defalt] times [1 + the remainder of the floorplate size minus the sum of all the optimzed room sizes]

In [5]:

solver.Minimize(
    (abs(1-s_patient_room.solution_value()/d_patient_room)
    + abs(1-s_ada_room.solution_value()/d_ada_room)
    + abs(1-s_supply.solution_value()/d_supply)
    + abs(1-s_soiled.solution_value()/d_soiled)
    + abs(1-s_nourish.solution_value()/d_nourish)
    + abs(1-s_alcove.solution_value()/d_alcove))
    * (1 +(34400 - (s_patient_room*n_patient_room + s_ada_room*n_ada_room + s_supply*n_supply + s_soiled*n_soiled + s_nourish*n_nourish + s_alcove*n_alcove)))
    )

finally, we'll optimize like so

In [6]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'solved in {solver.wall_time():.2f} miliseconds in {solver.iterations()} iterations')
    print()
    print(f'remainder: {34400 - (s_patient_room.solution_value()*n_patient_room + s_ada_room.solution_value()*n_ada_room + s_supply.solution_value()*n_supply + s_soiled.solution_value()*n_soiled + s_nourish.solution_value()*n_nourish + s_alcove.solution_value()*n_alcove)} sqft')
    print('rooms:')
    print(f' - {n_patient_room} patient_rooms at {s_patient_room.solution_value()/d_patient_room} * default size, for {s_patient_room.solution_value()} sqft each')
    print(f' - {n_ada_room} ada_rooms at {s_ada_room.solution_value()/d_ada_room} * default size, for {s_ada_room.solution_value()} sqft each')
    print(f' - {n_supply} supply at {s_supply.solution_value()/d_supply} * default size, for {s_supply.solution_value()} sqft each')
    print(f' - {n_soiled} soiled at {s_soiled.solution_value()/d_soiled} * default size, for {s_soiled.solution_value()} sqft each')
    print(f' - {n_nourish} norish at {s_nourish.solution_value()/d_nourish} * default size, for {s_nourish.solution_value()} sqft each')
    print(f' - {n_alcove} alcove at {s_alcove.solution_value()/d_alcove} * default size, for {s_alcove.solution_value()} sqft each')
else:
    print('the solver could not find an optimal solution')

================= Solution =================
solved in 146525.00 miliseconds in 0 iterations

remainder: 0.0 sqft
rooms:
 - 37 patient_rooms at 0.987969053542824 * default size, for 783.4594594594595 sqft each
 - 3 ada_rooms at 1.0495689655172413 * default size, for 974.0 sqft each
 - 2 supply at 1.0507399577167018 * default size, for 497.0 sqft each
 - 2 soiled at 1.0503778337531486 * default size, for 417.0 sqft each
 - 2 norish at 1.050420168067227 * default size, for 250.0 sqft each
 - 2 alcove at 1.051948051948052 * default size, for 81.0 sqft each
